# Classroom Seating Planning Tool
This tool helps make seating plans for rooms. To solve a room:
* First install all the necessary packages below for the tool to run. 
* Next run the cell below to import them all.
* Add a viable room diagram jpg or png to the file location of the tool
* Adjust the parameters to match the room
* Run all remaining cells and follow the instructions provided in the window. Green buttons are steps you have completed whilered buttons are steps you have not. 

In [1]:
# load packages
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd
import networkx as nx
import math, itertools, os, shapely
from shapely.geometry import Polygon, Point

from scipy import ndimage as ndi
from skimage import feature, io
from skimage.transform import rotate
from skimage.feature import match_template, peak_local_max
import cv2
import random
from ortools.linear_solver import pywraplp as OR


In the cell below, modify the parameters to match your situation.
* Screen_Height and Screen_Width must be equalto or smaller than your screen's dimensions or the window size will be larger than the screen.
* Floor must be the name of a to-scale classroom diagram in the same file location as the tool.
* LengthOfScaleInUnits must be the length of the scale of the diagram in terms of the units used when specifying how far apart chairs need to be.
* DistanceInUnitsForSocialDistancing must be the distance in the same units as LengthOfScaleInUnits that the chairs in a room must be apart.
* ScaleOrientation is the orientation of the scale of the diagram.
* Chair_Scale is the factor by which to scale up the image of each chair orientation when designating their shape.
* solution_name is for defining what you want the name of the classroom's seating solution to be.
* finding_threshold is the threshold the program uses in order to declare similarity between a chair orientation and a part of the image. If it is too low, the tool will detect chairs in places where they are not present, while a value that is too high will result in some chairs not being designated. finding_threshold has a maximum of 1 and a minimum of 0.
* Show_Instructions dictates whether or not instructions are shown. Set it to False to disable them.


In [2]:
#Window Configs:
Screen_Height=880 #The Height of Your Physical Screen In Pixels
Screen_Width=1920 #The Width of Your Physical Screen In Pixels

#Image Designation:
floor = 'ExampleRoom.png' # you need the path to the image file on YOUR computer

#Scale Selection Configs:
LengthOfScaleInUnits=20 #How many units long the scale is
DistanceInUnitsForSocialDistancing=6 #How many units apart the guidelines for social distancing
ScaleOrientation="Horizontal" #"Horizontal" or "Vertical"

#Polygon Creation Configs:
Chair_Scale=10 #The amount by which to scale up chairs when defining their shapes.

#Solution Configs:
solution_name="ExampleRoomSolution" #The name of the solution image

#Chair Recognition Configs:
finding_threshold=0.6 #The finding threshold of chairs.

#Show Instructions:
Show_Instructions=True

At this point, run all remaining cells in the jupyter notebook. In the menu that pops up, clicking in a green or red rectangular button will bring you to the corresponding input screen. Green buttons indicate input methods that have already been completed, while red buttons indicate input methods that have yet to be completed. Completed input methods can be rerun to re-do their input portion, but this will result in the mandatory re-doing of all input methods that have it as a prerequisite. The tool will add more input method buttons as you complete their prerequisites. 

In [3]:
#Program constant variables:
WindowName="Seating Planner"

DotSize=1

ScaleSelectionButtonStartPoint=(470,65)
ScaleSelectionButtonEndPoint=(670,115)
ScaleSelectionButtonBL=(75,100)
ScaleSelectionInstructionTitleBL=(750,100)
ScaleSelectionInstructionLine1BL=(270,150)
ScaleSelectionInstructionLine2BL=(270,175)
ScaleSelectionInstructionLine3BL=(270,200)
ScaleSelectionInstructionLine4BL=(270,225)

ChairTypeSelectionButtonStartPoint=(470,165)
ChairTypeSelectionButtonEndPoint=(670,215)
ChairTypeSelectionButtonBL=(75,200)
ChairTypeSelectionInstructionTitleBL=(670,100)
ChairTypeSelectionInstructionLine1BL=(270,150)
ChairTypeSelectionInstructionLine2BL=(270,175)
ChairTypeSelectionInstructionLine3BL=(270,200)
ChairTypeSelectionInstructionLine4BL=(270,225)
ChairTypeSelectionInstructionLine5BL=(270,250)

PolygonCreationSelectionButtonStartPoint=(470,265)
PolygonCreationSelectionButtonEndPoint=(670,315)
PolygonCreationSelectionButtonBL=(75,300)
PolygonCreationInstructionTitleBL=(700,100)
PolygonCreationInstructionLine1BL=(135,150)
PolygonCreationInstructionLine2BL=(135,175)
PolygonCreationInstructionLine3BL=(135,200)
PolygonCreationInstructionLine4BL=(135,225)
PolygonCreationInstructionLine5BL=(135,250)
PolygonCreationInstructionLine6BL=(135,275)
PolygonCreationInstructionLine7BL=(135,300)
PolygonCreationInstructionLine8BL=(135,325)

ChairRecognitionSelectionButtonStartPoint=(470,365)
ChairRecognitionSelectionButtonEndPoint=(670,415)
ChairRecognitionSelectionButtonBL=(75,400)
ChairRecognitionInstructionTitleBL=(700,100)
ChairRecognitionInstructionLine1BL=(135,150)
ChairRecognitionInstructionLine2BL=(135,175)
ChairRecognitionInstructionLine3BL=(135,200)
ChairRecognitionInstructionLine4BL=(135,225)
ChairRecognitionInstructionLine5BL=(135,250)
ChairRecognitionInstructionLine6BL=(135,275)
ChairRecognitionInstructionLine7BL=(135,300)
ChairRecognitionInstructionLine8BL=(135,325)
ChairRecognitionInstructionLine9BL=(135,350)

ChairAdditionSelectionButtonStartPoint=(470,465)
ChairAdditionSelectionButtonEndPoint=(670,515)
ChairAdditionSelectionButtonBL=(75,500)
ChairAdditionInstructionTitleBL=(770,100)
ChairAdditionInstructionLine1BL=(535,150)
ChairAdditionInstructionLine2BL=(535,175)
ChairAdditionInstructionLine3BL=(535,200)
ChairAdditionInstructionLine4BL=(535,225)
ChairAdditionInstructionLine5BL=(535,250)
ChairAdditionInstructionLine6BL=(535,275)
ChairAdditionInstructionLine7BL=(535,300)
ChairAdditionInstructionLine8BL=(535,325)
ChairAdditionInstructionLine9BL=(535,350)

ChairDeletionSelectionButtonStartPoint=(470,565)
ChairDeletionSelectionButtonEndPoint=(670,615)
ChairDeletionSelectionButtonBL=(75,600)
ChairDeletionInstructionTitleBL=(770,100)
ChairDeletionInstructionLine1BL=(535,150)
ChairDeletionInstructionLine2BL=(535,175)
ChairDeletionInstructionLine3BL=(535,200)
ChairDeletionInstructionLine4BL=(535,225)
ChairDeletionInstructionLine5BL=(535,250)
ChairDeletionInstructionLine6BL=(535,275)
ChairDeletionInstructionLine7BL=(535,300)
ChairDeletionInstructionLine8BL=(535,325)

InputPreviewButtonStartPoint=(470,665)
InputPreviewButtonEndPoint=(670,715)
InputPreviewButtonBL=(75,700)
InputPreviewInstructionTitleBL=(800,100)
InputPreviewInstructionLine1BL=(435,150)
InputPreviewInstructionLine2BL=(435,175)
InputPreviewInstructionLine3BL=(435,200)
InputPreviewInstructionLine4BL=(435,225)
InputPreviewInstructionLine5BL=(435,250)

SolveAndSaveButtonStartPoint=(470,765)
SolveAndSaveButtonEndPoint=(670,815)
SolveAndSaveButtonBL=(75,800)
SolveRoomInstructionTitleBL=(800,100)
SolveRoomInstructionLine1BL=(385,150)
SolveRoomInstructionLine2BL=(385,175)
SolveRoomInstructionLine3BL=(385,200)
SolveRoomInstructionLine4BL=(385,225)
SolveRoomInstructionLine5BL=(385,250)

NotCompletedXAdjustment=12
CompletedXAdjustment=36

ButtonThickness=5
ButtonNotCompleteColor=(0,0,255)
ButtonCompleteColor=(0,255,0)

Green=(0,255,0)
Purple=(255,0,255)
Red=(0,0,255)

In [4]:
floorU = cv2.imread(floor, cv2.IMREAD_UNCHANGED)
floor_str = floor

#REQUIRED INFO. THESE VARIABLES WILL BE UPDATED BY THE USER AS THEY INPUT THE CLASSROOM DETAILS IN THE PROGRAM
RoomInfo={"PixelsToDistance":"Not Set",
          "ScaleSelectedPoints":[],
          "ChairTypes":"Not Set",
          "Polys":"Not Set",
          "PolyPoints":[],
          "PolyPointOffsets":[],
          "Locs":"Not Set"}

Completions={"Scale Selection Status":False,
             "Chair Type Selection Status":False,
             "Polygon Creation Status":False,
             "Chair Recognition Status":False,
             "Chair Addition Status":False,
             "Chair Deletion Status":False,
             "Input Confirmation Status":False}

Doorways=[]

In [5]:
if floorU.shape[0] > Screen_Height:
    height=floorU.shape[0]
    Window_Height=Screen_Height
else:
    height=Screen_Height
    Window_Height=floorU.shape[0]-1
if floorU.shape[1]>Screen_Width:
    width=floorU.shape[1]
    Window_Width=Screen_Width
else:
    width=Screen_Width
    Window_Width=floorU.shape[1]-1
NonWritableImage=cv2.imread(floor)
WritableImage=cv2.imread(floor)

In [6]:
# get lh, .., uw from full set of 4 coordinates
def getCoords(rect):
    """
    Takes an np.array with coords of rectangle.
    Returns max / min height / width
    """
    lw = min(rect[:,0])
    uw = max(rect[:,0])
    lh = min(rect[:,1])
    uh = max(rect[:,1])

    return(lw, uw, lh, uh)

In [7]:
# a model for counting seats (function by Jody Zhu)

# 'nodes' is a list of chairs, 'edges' is a list of (chair,chair) tuples that are too close
def miset(nodes,edges,solver):
    NODES = []
    NODES.extend(nodes)    # these four lines are not necessary
    EDGES = []             # made a copy in case the lists will be modified
    EDGES.extend(edges)
    
    # define model
    m = OR.Solver('maxIndSet', solver)
    
    # decision variables
    x = {} # node i is in the maximal independent set is 1, else 0
    for i in NODES:
        x[i] = m.IntVar(0, 1, ('%s' % (i))) 
        
    # objective function
    m.Maximize(sum(x[i] for i in NODES))
        
    # subject to: no more than 1 node from an edge
    for edge in EDGES:
        m.Add(x[edge[0]] + x[edge[1]] <= 1)

    m.Solve()
    
    sol = {}
    for i in NODES:
        sol.update( {x[i].name() : x[i].solution_value()} )

    
    return(sol)

In [8]:
def TrackbarChange(integer):
     return

In [9]:
cv2.namedWindow(WindowName,cv2.WINDOW_AUTOSIZE)
cv2.resizeWindow(WindowName,Window_Width, Window_Height)

In [10]:
def ScaleSelectionExplanation():
    GeneratedImage=np.ones((Screen_Height,Screen_Width,1),np.uint8)*255
    GeneratedImage=cv2.cvtColor(GeneratedImage,cv2.COLOR_GRAY2RGB)
    GeneratedImage=cv2.putText(GeneratedImage,"Scale Selection",ScaleSelectionInstructionTitleBL,cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Left click two diagonal corners of the room diagram's scale.",ScaleSelectionInstructionLine1BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Right click to cancel your previous click.",ScaleSelectionInstructionLine2BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"When you are done, make sure the selection rectangle is purple. Press 'e' to finalize your scale selection.",ScaleSelectionInstructionLine3BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Press 'e' to continue to the scale selection screen.",ScaleSelectionInstructionLine4BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)

    while True:
        cv2.imshow(WindowName,GeneratedImage)
        key=cv2.waitKey(1)&0xFF
        if key==ord("e"):
            break
    return
        
#Define the Scale
def ScaleSelection(NonWritableImage,menuRefresher):
    if Show_Instructions:
        ScaleSelectionExplanation()
    scrollHeight=0
    scrollWidth=0
    WritableClone=NonWritableImage.copy()
    ref=[]
    refTemp=[]
    refresh=[]

    cv2.createTrackbar("Hscroll", WindowName, scrollHeight, (height-Screen_Height),TrackbarChange)
    cv2.createTrackbar("Wscroll", WindowName, scrollWidth, (width-Screen_Width),TrackbarChange)

    def get_scale_selection_click_coords(event, x, y, flags, param):
        if event == cv2.EVENT_MOUSEMOVE:
            for item in range(len(refTemp)):
                del refTemp[-1]
            refTemp.append((x+cv2.getTrackbarPos("Wscroll", WindowName), y+cv2.getTrackbarPos("Hscroll", WindowName)))
            refresh.append('r')
        if event == cv2.EVENT_LBUTTONDOWN:
            if len(ref)<2:
                ref.append((x+cv2.getTrackbarPos("Wscroll", WindowName), y+cv2.getTrackbarPos("Hscroll", WindowName)))
                refresh.append('r')
        elif event==cv2.EVENT_RBUTTONDOWN:
            if len(ref)>0:
                del ref[-1]
                refresh.append('r')

    cv2.setMouseCallback(WindowName, get_scale_selection_click_coords)

    def finalize_scale(orientation,stickLength,distancingLength):
        assert(orientation=="Vertical" or orientation=="Horizontal")
        x1=ref[0][0]
        y1=ref[0][1]
        x2=ref[1][0]
        y2=ref[1][1]
        if x1>x2:
            x1temp=x1
            x1=x2
            x2=x1temp
        if y1>y2:
            y1temp=y1
            y1=y2
            y2=y1temp
        RoomInfo["ScaleSelectedPoints"]=[(x1, y1), (x2, y1), (x2, y2), (x1, y2)]
        if orientation=="Vertical":
            stickLengthPixels=y2-y1
        else:
            stickLengthPixels=x2-x1
        oneUnit=stickLengthPixels/stickLength
        distancingPixels=distancingLength*oneUnit
        cv2.destroyAllWindows()
        return distancingPixels

        
    while True:
    
        #These lines are what allow the scrolling
        ScrollHeight=cv2.getTrackbarPos("Hscroll", WindowName)
        ScrollWidth=cv2.getTrackbarPos("Wscroll", WindowName)
        ShownImage = np.array([(ScrollWidth,ScrollHeight),
                               (ScrollWidth+width,ScrollHeight),
                               (ScrollWidth+width,ScrollHeight+height),
                               (ScrollWidth,ScrollHeight+height)])
        cds = getCoords(ShownImage)
        Image = WritableClone[cds[2]:cds[3], cds[0]:cds[1]]
        cv2.imshow(WindowName, Image)
    
        #Refreshes the image if there has been an update to it
        if len(refresh)>=1:
            WritableClone=NonWritableImage.copy()
            if len(ref)>=2:
                for i in range((len(ref)-(len(ref)%2))//2):
                    cv2.rectangle(WritableClone,ref[i*2],ref[i*2+1],Purple, 1)
            if len(ref)%2==1:
                cv2.rectangle(WritableClone,ref[-1],refTemp[0],Green,1)
            refresh=[]
    
        #Listen for key presses
        key = cv2.waitKey(1) & 0xFF
    
        # if the 'e' key is pressed, break from the loop
        if key == ord("e"):
            if (len(ref)==2):
                menuRefresher.append('r')
                RoomInfo["PixelsToDistance"]=finalize_scale(ScaleOrientation,
                                                            LengthOfScaleInUnits,
                                                            DistanceInUnitsForSocialDistancing)
                break



In [11]:
def ChairTypeSelectionExplanation():
    GeneratedImage=np.ones((Screen_Height,Screen_Width,1),np.uint8)*255
    GeneratedImage=cv2.cvtColor(GeneratedImage,cv2.COLOR_GRAY2RGB)
    GeneratedImage=cv2.putText(GeneratedImage,"Chair Type Selection",ChairTypeSelectionInstructionTitleBL,cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Left click two diagonal corners to define a rectangle bounding each unique chair orientation in the room",ChairTypeSelectionInstructionLine1BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Right click to cancel your previous click.",ChairTypeSelectionInstructionLine2BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Do this for each unique chair shape and rotation in the image.",ChairTypeSelectionInstructionLine3BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Make sure that each chair orientation is bounded by a purple rectangle before pressing 'e' to finalize your selections.",ChairTypeSelectionInstructionLine4BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Press 'e' to continue to the chair type selection screen.",ChairTypeSelectionInstructionLine5BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)

    while True:
        cv2.imshow(WindowName,GeneratedImage)
        key=cv2.waitKey(1)&0xFF
        if key==ord("e"):
            break
    return

def ChairTypeSelection(NonWritableImage, menuRefresher):
    if Show_Instructions:
        ChairTypeSelectionExplanation()
    scrollHeight=0
    scrollWidth=0
    WritableClone=NonWritableImage.copy()
    ref=[]
    refTemp=[]
    refresh=[]

    cv2.createTrackbar("Hscroll", WindowName, scrollHeight, (height-Screen_Height),TrackbarChange)
    cv2.createTrackbar("Wscroll", WindowName, scrollWidth, (width-Screen_Width),TrackbarChange)
    
    def get_ChairType_selection_click_coords(event, x, y, flags, param):
        if event == cv2.EVENT_MOUSEMOVE:
            for item in range(len(refTemp)):
                del refTemp[-1]
            refTemp.append((x+cv2.getTrackbarPos("Wscroll", WindowName), y+cv2.getTrackbarPos("Hscroll", WindowName)))
            refresh.append('r')
        if event == cv2.EVENT_LBUTTONDOWN:
            ref.append((x+cv2.getTrackbarPos("Wscroll", WindowName), y+cv2.getTrackbarPos("Hscroll", WindowName)))
            refresh.append('r')
        elif event==cv2.EVENT_RBUTTONDOWN:
            if len(ref)>0:
                del ref[-1]
                refresh.append('r')
                
    cv2.setMouseCallback(WindowName, get_ChairType_selection_click_coords)
    
    def Finalize_Chair_Types():
        chairs=[]
        for r in range(len(ref)//2):
            x1 = ref[2*r][0]
            y1 = ref[2*r][1]
            x2 = ref[2*r+1][0]
            y2 = ref[2*r+1][1]
            ch = np.array([(x1, y1), (x2, y1), (x2, y2), (x1, y2)])
            chairs.append(ch)
        return chairs
    while True:
    
        #These lines are what allow the scrolling
        ScrollHeight=cv2.getTrackbarPos("Hscroll", WindowName)
        ScrollWidth=cv2.getTrackbarPos("Wscroll", WindowName)
        ShownImage = np.array([(ScrollWidth,ScrollHeight),
                               (ScrollWidth+width,ScrollHeight),
                               (ScrollWidth+width,ScrollHeight+height),
                               (ScrollWidth,ScrollHeight+height)])
        cds = getCoords(ShownImage)
        Image = WritableClone[cds[2]:cds[3], cds[0]:cds[1]]
        cv2.imshow(WindowName, Image)
    
        #Refreshes the image if there has been an update to it
        if len(refresh)>=1:
            WritableClone=NonWritableImage.copy()
            if len(ref)>=2:
                for i in range((len(ref)-(len(ref)%2))//2):
                    cv2.rectangle(WritableClone,ref[i*2],ref[i*2+1],Purple, 1)
            if len(ref)%2==1:
                cv2.rectangle(WritableClone,ref[-1],refTemp[0],Green,1)
            refresh=[]
    
        #Listen for key presses
        key = cv2.waitKey(1) & 0xFF
    
        # if the 'e' key is pressed, break from the loop
        if key == ord("e"):
            if (len(ref)%2==0):
                menuRefresher.append('r')
                RoomInfo["ChairTypes"]=Finalize_Chair_Types()
                break

    
    

In [12]:
def PolygonCreationExplanation():
    GeneratedImage=np.ones((Screen_Height,Screen_Width,1),np.uint8)*255
    GeneratedImage=cv2.cvtColor(GeneratedImage,cv2.COLOR_GRAY2RGB)
    GeneratedImage=cv2.putText(GeneratedImage,"Polygon Creation",PolygonCreationInstructionTitleBL,cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Left click points on the perimeter of each chair to add them to that chair's polygon.",PolygonCreationInstructionLine1BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Right click to cancel your previous click.",PolygonCreationInstructionLine2BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"You must create a polygon for each chair orientation selected in the chair type selection.",PolygonCreationInstructionLine3BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"These polygons define the shape of each chair, so it is important to define them accurately.",PolygonCreationInstructionLine4BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Order matters between points clicked, so click the points around the edge in order as if you were tracing around the chair's shape.",PolygonCreationInstructionLine5BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Press 'e' to finalize the polygon for the current chair and advance to the next one.",PolygonCreationInstructionLine6BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Pressing 'e' on the final polygon creation screen will finalize that polygon and send you back to the menu.",PolygonCreationInstructionLine7BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Press 'e' to proceed to the first polygon creation screen.",PolygonCreationInstructionLine8BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)

    while True:
        cv2.imshow(WindowName,GeneratedImage)
        key=cv2.waitKey(1)&0xFF
        if key==ord("e"):
            break
    return
def PolygonCreation(NonWritableImage,menuRefresher,RoomInfo):
    if Show_Instructions:
        PolygonCreationExplanation()
    #Create Polygons for each selected chair type
    polygons={}
    xoffsets={}
    yoffsets={}
    Polygons=[]#The list of polygons in the same order as chairtype
    RoomInfo['PolyPoints']=[]
    RoomInfo['PolyPointOffsets']=[]
    for i in range(len(RoomInfo["ChairTypes"])):#for each chair
        cds = getCoords(RoomInfo["ChairTypes"][i])
        template = NonWritableImage[cds[2]:cds[3], cds[0]:cds[1]]#template is the subsection of img that contains the chair
        height=len(template)#gets the height of the subsection
        width=len(template[0])#gets the width of the subsection
        xoffsets[i]=-width/2
        yoffsets[i]=-height/2
        ref=[]#creates array ref[] that is used to store all points that will be added to the polygon
        template=cv2.resize(template,(Chair_Scale*width,Chair_Scale*height))#resizes the image to be 4 times the original size
        templateCopy=template.copy()
        refresh=[]
        def polygon_creation(event, x, y, flags, param):#polygon_creation is the function called whenever a mouse event occurs
            if event == cv2.EVENT_LBUTTONDOWN:#If the left mouse button is clicked, add the point (scaled down to the chair's original size) to ref[].
                ref.append((int(x/Chair_Scale),int(y/Chair_Scale)))
                refresh.append('r')
            elif event == cv2.EVENT_RBUTTONDOWN:   # right-click to delete the latest point added to ref from the polygon
                del ref[-1]
                refresh.append('r')
        cv2.setMouseCallback(WindowName, polygon_creation)
        while True:
            cv2.imshow(WindowName,templateCopy)
            if len(refresh)>=1:
                templateCopy=template.copy()
                points=len(ref)
                if points>=2:#if there are at least 2 points, adds a line between the latest created point and the previous point.
                #this allows the polygon to be drawn by the user on the window as they are marking it out
                    for PolyPoint in range(points):
                        line=cv2.line(templateCopy,
                                      (ref[PolyPoint][0]*Chair_Scale,ref[PolyPoint][1]*Chair_Scale),
                                      (ref[PolyPoint-1][0]*Chair_Scale,ref[PolyPoint-1][1]*Chair_Scale),
                                      Red,1)
                for PolyPoint in range(points):
                    dot=cv2.circle(templateCopy,(ref[PolyPoint][0]*Chair_Scale,ref[PolyPoint][1]*Chair_Scale),int(DotSize*Chair_Scale/2),Red,-1)
                cv2.imshow(WindowName, templateCopy)#show template in the window
                refresh=[]
            key = cv2.waitKey(1) & 0xFF
            if key==ord('e'):#close the window if 'e' is pressed
                if len(ref)>=3:
                    break
        Polygons.append(Polygon(np.array(ref)))#adds the Polygon to Polygons[]
        RoomInfo["PolyPoints"].append(ref)
        RoomInfo["PolyPointOffsets"].append([int(-width/2),int(-height/2)])
    for polygon in range(len(Polygons)):
        polygons[polygon]=shapely.affinity.rotate(Polygons[polygon],0, origin='center')
    menuRefresher.append('r')
    RoomInfo['Polys']=polygons

In [13]:
def ChairRecognitionExplanation():
    GeneratedImage=np.ones((Screen_Height,Screen_Width,1),np.uint8)*255
    GeneratedImage=cv2.cvtColor(GeneratedImage,cv2.COLOR_GRAY2RGB)
    GeneratedImage=cv2.putText(GeneratedImage,"Chair Recognition",ChairRecognitionInstructionTitleBL,cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Chair recognition does not require user input.",ChairRecognitionInstructionLine1BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"This step can take a long time depending on the size of the room and/or how many chairs it is detecting.",ChairRecognitionInstructionLine2BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,'This step will often show up as "Not Responding" even though it is working.',ChairRecognitionInstructionLine3BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"When completed, it will show chairs highlighted within the room. ",ChairRecognitionInstructionLine4BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Press 'e' at that point to confirm the completion of Chair Recognition.",ChairRecognitionInstructionLine5BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Remember, the results of Chair Recognition can be adjusted in the following steps, so it is not necessary to have perfect results.",ChairRecognitionInstructionLine6BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"If a substantial amount of chairs are being placed erroneously, restart the tool with a higher finding_threshold.",ChairRecognitionInstructionLine7BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Similarly, if there are way too many chairs that failed to be recognized, restrt the tool with a lower finding_threshold.",ChairRecognitionInstructionLine8BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Press 'e' to proceed to chair recognition.",ChairRecognitionInstructionLine9BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    while True:
        cv2.imshow(WindowName,GeneratedImage)
        key=cv2.waitKey(1)&0xFF
        if key==ord("e"):
            break
    return

def ChairRecognition(NonWritableImage,menuRefresher,chairs):
    if Show_Instructions:
        ChairRecognitionExplanation()
    
    scrollHeight=0
    scrollWidth=0
    
    cv2.createTrackbar("Hscroll", WindowName, scrollHeight, (height-Screen_Height),TrackbarChange)
    cv2.createTrackbar("Wscroll", WindowName, scrollWidth, (width-Screen_Width),TrackbarChange)
    
    # initialize empty array
    x = np.array([])
    y = np.array([])
    ctype = np.array([])
    
    WritableClone=NonWritableImage.copy()
    # extract all types of chairs
    for i in range(len(chairs)):
        cds = getCoords(chairs[i])
        chair = NonWritableImage[cds[2]:cds[3], cds[0]:cds[1]] # get chair template
    
        # run matching function (may take a minute or two)
        result = match_template(NonWritableImage, chair, pad_input=True)
        points = peak_local_max(result, min_distance=1, threshold_rel=finding_threshold) # find our peaks (our location points)
    
        # add points to list
        x = np.append(x, points[:,1])
        y = np.append(y, points[:,0])
        ctype = np.append(ctype, np.repeat(i, len(points)))
    
        # add points to cv2
        for point in range(len(points)):
            center_coordinates = (points[point,1], points[point,0])
            x_center=center_coordinates[0]
            y_center=center_coordinates[1]
            PolygonPoints=RoomInfo['PolyPoints'][i]
            PolygonOffsets=RoomInfo['PolyPointOffsets'][i]
            for polygonPoint in range(len(PolygonPoints)):
                line=cv2.line(WritableClone,
                              (PolygonPoints[polygonPoint][0]+x_center+PolygonOffsets[0],
                               PolygonPoints[polygonPoint][1]+y_center+PolygonOffsets[1]),
                              (PolygonPoints[polygonPoint-1][0]+x_center+PolygonOffsets[0],
                               PolygonPoints[polygonPoint-1][1]+y_center+PolygonOffsets[1]),
                              Red,1)
            cv2.circle(WritableClone, center_coordinates, 2, Red, 2)
    
    while True:
    #These lines are what allow the scrolling
        ScrollHeight=cv2.getTrackbarPos("Hscroll", WindowName)
        ScrollWidth=cv2.getTrackbarPos("Wscroll", WindowName)
        ShownImage = np.array([(ScrollWidth,ScrollHeight),
                               (ScrollWidth+width,ScrollHeight),
                               (ScrollWidth+width,ScrollHeight+height),
                               (ScrollWidth,ScrollHeight+height)])
        cds = getCoords(ShownImage)
        Image = WritableClone[cds[2]:cds[3], cds[0]:cds[1]]
        cv2.imshow(WindowName, Image)
        key=cv2.waitKey(1)&0xFF
        if key==ord('e'):#close the window if 'e' is pressed
            break
        
    
    
    # store output in RoomInfo
    menuRefresher.append('r')
    xarray=[]
    for xcoordinate in x:
        xarray.append(xcoordinate)
    yarray=[]
    for ycoordinate in y:
        yarray.append(ycoordinate)
    
    ctypearray=[]
    for chairtypeint in ctype:
        ctypearray.append(chairtypeint)
    RoomInfo['Locs']={'x':xarray,'y':yarray,'type':ctypearray}

In [14]:
def ChairAdditionExplanation():
    GeneratedImage=np.ones((Screen_Height,Screen_Width,1),np.uint8)*255
    GeneratedImage=cv2.cvtColor(GeneratedImage,cv2.COLOR_GRAY2RGB)
    GeneratedImage=cv2.putText(GeneratedImage,"Chair Addition",ChairAdditionInstructionTitleBL,cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Press 'd' to cycle to the next chair type",ChairAdditionInstructionLine1BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Press 'a' to cycle to the previous chair type",ChairAdditionInstructionLine2BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,'A preview chair will be shown in green around your cursor.',ChairAdditionInstructionLine3BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Left click to confirm placement of the preview chair.",ChairAdditionInstructionLine4BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"At this point, the chair should turn purple indicating its placement.",ChairAdditionInstructionLine5BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"You can place multiple chairs during this step.",ChairAdditionInstructionLine6BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Right click to undo your most recent chair placement.",ChairAdditionInstructionLine7BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Press 'e' to confirm all your placed chairs and return to the menu",ChairAdditionInstructionLine8BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Press 'e' to proceed to chair addition.",ChairAdditionInstructionLine9BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    while True:
        cv2.imshow(WindowName,GeneratedImage)
        key=cv2.waitKey(1)&0xFF
        if key==ord("e"):
            break
    return
def ChairAddition(NonWritableImage,menuRefresher,chairs):
    if Show_Instructions:
        ChairAdditionExplanation()
    scrollHeight=0
    scrollWidth=0
    
    cv2.createTrackbar("Hscroll", WindowName, scrollHeight, (height-Screen_Height),TrackbarChange)
    cv2.createTrackbar("Wscroll", WindowName, scrollWidth, (width-Screen_Width),TrackbarChange)
    
    chairSelection=0
    chairTypes=len(chairs)-1
    WritableClone=NonWritableImage.copy()
    
    ref=[]
    refTemp=[]
    refresh=['r']
    
    def place_Chair(event,x,y,flags,param):
        if event==cv2.EVENT_MOUSEMOVE:
            for item in range(len(refTemp)):
                del refTemp[-1]
            refTemp.append([x+cv2.getTrackbarPos("Wscroll", WindowName),
                            y+cv2.getTrackbarPos("Hscroll",WindowName),
                            chairSelection])
            refresh.append('r')
        if event==cv2.EVENT_LBUTTONDOWN:
            ref.append([x+cv2.getTrackbarPos("Wscroll", WindowName),
                        y+cv2.getTrackbarPos("Hscroll",WindowName),
                        chairSelection])
            refresh.append('r')
        if event==cv2.EVENT_RBUTTONDOWN:
            del ref[-1]
            refresh.append('r')
            
    cv2.setMouseCallback(WindowName,place_Chair)
    
    while True:
        ScrollHeight=cv2.getTrackbarPos("Hscroll",WindowName)
        ScrollWidth=cv2.getTrackbarPos("Wscroll",WindowName)
        ShownImage=np.array([(ScrollWidth,ScrollHeight),
                             (ScrollWidth+width,ScrollHeight),
                             (ScrollWidth+width,ScrollHeight+height),
                             (ScrollWidth,ScrollHeight+height)])
        cds=getCoords(ShownImage)
        Image = WritableClone[cds[2]:cds[3], cds[0]:cds[1]]
        cv2.imshow(WindowName, Image)
        if len(refresh)>=1:
            WritableClone=NonWritableImage.copy()
            #convert RoomInfo['Locs']
            xcoords=[]
            ycoords=[]
            chairtypes=[]
            for xcoord in RoomInfo['Locs']['x']:
                xcoords.append(int(xcoord))
            for ycoord in RoomInfo['Locs']['y']:
                ycoords.append(int(ycoord))
            for chairtype in RoomInfo['Locs']['type']:
                chairtypes.append(int(chairtype))

            #Re-draw existing chairs
            for chair in range(len(xcoords)):
                x_center=xcoords[chair]
                y_center=ycoords[chair]
                PolygonOffsets=RoomInfo['PolyPointOffsets'][chairtypes[chair]]
                cv2.circle(WritableClone, (x_center,y_center), 2, Red,2)
                PolygonPoints=RoomInfo['PolyPoints'][chairtypes[chair]]
                for polygonPoint in range(len(PolygonPoints)):
                    line=cv2.line(WritableClone,
                                  (PolygonPoints[polygonPoint][0]+x_center+PolygonOffsets[0],
                                   PolygonPoints[polygonPoint][1]+y_center+PolygonOffsets[1]),
                                  (PolygonPoints[polygonPoint-1][0]+x_center+PolygonOffsets[0],
                                   PolygonPoints[polygonPoint-1][1]+y_center+PolygonOffsets[1]),
                                  Red,1)
            
            #Draw Temp chair
            if len(refTemp)>=1:
                PolygonOffsets=RoomInfo['PolyPointOffsets'][refTemp[0][2]]
                PolygonPoints=RoomInfo['PolyPoints'][refTemp[0][2]]
                x_center=refTemp[0][0]
                y_center=refTemp[0][1]
                for polygonPoint in range(len(PolygonPoints)):
                    line=cv2.line(WritableClone,
                                  (PolygonPoints[polygonPoint][0]+x_center+PolygonOffsets[0],
                                   PolygonPoints[polygonPoint][1]+y_center+PolygonOffsets[1]),
                                  (PolygonPoints[polygonPoint-1][0]+x_center+PolygonOffsets[0],
                                   PolygonPoints[polygonPoint-1][1]+y_center+PolygonOffsets[1]),
                                  Green,1)
            
            for chair in ref:
                PolygonOffsets=RoomInfo['PolyPointOffsets'][chair[2]]
                PolygonPoints=RoomInfo['PolyPoints'][chair[2]]
                x_center=chair[0]
                y_center=chair[1]
                for polygonPoint in range(len(PolygonPoints)):
                    line=cv2.line(WritableClone,
                                  (PolygonPoints[polygonPoint][0]+x_center+PolygonOffsets[0],
                                   PolygonPoints[polygonPoint][1]+y_center+PolygonOffsets[1]),
                                  (PolygonPoints[polygonPoint-1][0]+x_center+PolygonOffsets[0],
                                   PolygonPoints[polygonPoint-1][1]+y_center+PolygonOffsets[1]),
                                  Purple,1) 
            
            refresh=[]
                
        key=cv2.waitKey(1)&0xFF
        if key==ord('e'): #close the window if 'e' is pressed
            break
        elif key==ord('d'): #shift to the next chair type if key 'a' is pressed
            if chairSelection<chairTypes:
                chairSelection=chairSelection+1
            else:
                chairSelection=0
        elif key==ord('a'): #shift to the previous chair type if key 'd' is pressed
            if chairSelection>0:
                chairSelection=chairSelection-1
            else:
                chairSelection=chairTypes
    startloc=len(RoomInfo['Locs'])
    for chair in range(len(ref)):
        RoomInfo['Locs']['x'].append(ref[chair][0])
        RoomInfo['Locs']['y'].append(ref[chair][1])
        RoomInfo['Locs']['type'].append(ref[chair][2])
        
    menuRefresher.append('r')

In [15]:
def ChairDeletionExplanation():
    GeneratedImage=np.ones((Screen_Height,Screen_Width,1),np.uint8)*255
    GeneratedImage=cv2.cvtColor(GeneratedImage,cv2.COLOR_GRAY2RGB)
    GeneratedImage=cv2.putText(GeneratedImage,"Chair Deletion",ChairDeletionInstructionTitleBL,cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Left click to select a point.",ChairDeletionInstructionLine1BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Left click again to define a rectangle between the two clicked points.",ChairDeletionInstructionLine2BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"This rectangle should be purple when completed correctly.",ChairDeletionInstructionLine3BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"All chairs within the rectangle will be deleted upon completion.",ChairDeletionInstructionLine4BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Multiple rectangles can be defined.",ChairDeletionInstructionLine5BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Right click to undo the previously selected rectangle point.",ChairDeletionInstructionLine6BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Press 'e' to confirm all your deleted chairs and return to the menu",ChairDeletionInstructionLine7BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Press 'e' to proceed to chair deletion.",ChairDeletionInstructionLine8BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    while True:
        cv2.imshow(WindowName,GeneratedImage)
        key=cv2.waitKey(1)&0xFF
        if key==ord("e"):
            break
    return

def ChairDeletion(NonWritableImage,menuRefresher):
    if Show_Instructions:
        ChairDeletionExplanation()

    scrollHeight=0
    scrollWidth=0
    
    cv2.createTrackbar("Hscroll", WindowName, scrollHeight, (height-Screen_Height),TrackbarChange)
    cv2.createTrackbar("Wscroll", WindowName, scrollWidth, (width-Screen_Width),TrackbarChange)
    
    WritableClone=NonWritableImage.copy()
    WritableCloneBackup=WritableClone.copy()
    ref=[]
    refTemp=[]
    refresh=['r']
    
    Chairs=[]
    
    for chair in range(len(RoomInfo['Locs']['x'])):
        Chairs.append([int(RoomInfo['Locs']['x'][chair]),
                       int(RoomInfo['Locs']['y'][chair]),
                       int(RoomInfo['Locs']['type'][chair])])
        
    #Draw all chairs on a fresh room
    for chair in Chairs:
        cv2.circle(WritableClone,(chair[0],chair[1]),2,Red,2)
        PolygonPoints=RoomInfo['PolyPoints'][chair[2]]
        PolygonOffsets=RoomInfo['PolyPointOffsets'][chair[2]]
        for polygonPoint in range(len(PolygonPoints)):
            line=cv2.line(WritableClone,
                          (PolygonPoints[polygonPoint][0]+chair[0]+PolygonOffsets[0],
                           PolygonPoints[polygonPoint][1]+chair[1]+PolygonOffsets[1]),
                          (PolygonPoints[polygonPoint-1][0]+chair[0]+PolygonOffsets[0],
                           PolygonPoints[polygonPoint-1][1]+chair[1]+PolygonOffsets[1]),
                          Red,1)
    #Take a copy of the current drawing state
    WritableCloneBackup=WritableClone.copy()
    
    def Delete_Chairs(event,x,y,flags,param):
        if event==cv2.EVENT_MOUSEMOVE:
            for item in range(len(refTemp)):
                del refTemp[-1]
            refTemp.append((x+cv2.getTrackbarPos("Wscroll", WindowName),y+cv2.getTrackbarPos("Hscroll",WindowName)))
            refresh.append('r')
        if event==cv2.EVENT_LBUTTONDOWN:
            ref.append((x+cv2.getTrackbarPos("Wscroll", WindowName),y+cv2.getTrackbarPos("Hscroll",WindowName)))
            refresh.append('r')
        if event==cv2.EVENT_RBUTTONDOWN:
            del ref[-1]
            refresh.append('r')
    
    cv2.setMouseCallback(WindowName,Delete_Chairs)
    
    while True:
        ScrollHeight=cv2.getTrackbarPos("Hscroll",WindowName)
        ScrollWidth=cv2.getTrackbarPos("Wscroll",WindowName)
        ShownImage=np.array([(ScrollWidth,ScrollHeight),
                             (ScrollWidth+width,ScrollHeight),
                             (ScrollWidth+width,ScrollHeight+height),
                             (ScrollWidth,ScrollHeight+height)])
        cds=getCoords(ShownImage)
        Image = WritableClone[cds[2]:cds[3], cds[0]:cds[1]]
        cv2.imshow(WindowName, Image)
        
        #Refreshes the image
        if len(refresh)>=1:
            #Reverts to the previous drawing state copy with chairs drawn in.
            WritableClone=WritableCloneBackup.copy()
            
            #Draws selection Rectangles
            if len(ref)>=2:
                for i in range((len(ref)-(len(ref)%2))//2):
                    cv2.rectangle(WritableClone,ref[i*2],ref[i*2+1],Purple, 1)
            if len(ref)%2==1:
                cv2.rectangle(WritableClone,ref[-1],refTemp[0],Green,1)
            
            refresh=[]
            
        key=cv2.waitKey(1)&0xFF    
            
        if key==ord('e'):
            if len(ref)%2==0:
                break
                
    Rectangles=[]
    if len(ref)>=2:
        for i in range((len(ref)-len(ref)%2)//2):
            x_coord1=ref[i*2][0]
            y_coord1=ref[i*2][1]
            x_coord2=ref[i*2+1][0]
            y_coord2=ref[i*2+1][1]
            
            if x_coord1>x_coord2:
                x_temp=x_coord1
                x_coord1=x_coord2
                x_coord2=x_temp
                
            if y_coord1>y_coord2:
                y_temp=y_coord1
                y_coord1=y_coord2
                y_coord2=y_temp
            
            Rectangles.append([(x_coord1,y_coord1),(x_coord2,y_coord2)])
            
    for rectangle in Rectangles:
        removeThese=[]
        for chair in Chairs:
            if chair[0]>rectangle[0][0] and chair[0]<rectangle[1][0]:
                if chair[1]>rectangle[0][1] and chair[1]<rectangle[1][1]:
                    removeThese.append(chair)
                    
        while len(removeThese)>=1:
            Chairs.remove(removeThese[0])
            del removeThese[0]
    
    RoomInfo['Locs']['x'].clear()
    RoomInfo['Locs']['y'].clear()
    RoomInfo['Locs']['type'].clear()
    
    for chair in Chairs:
        RoomInfo['Locs']['x'].append(chair[0])
        RoomInfo['Locs']['y'].append(chair[1])
        RoomInfo['Locs']['type'].append(chair[2])
    
    menuRefresher.append('r')


In [16]:
def InputPreviewExplanation():
    GeneratedImage=np.ones((Screen_Height,Screen_Width,1),np.uint8)*255
    GeneratedImage=cv2.cvtColor(GeneratedImage,cv2.COLOR_GRAY2RGB)
    GeneratedImage=cv2.putText(GeneratedImage,"InputPreview",InputPreviewInstructionTitleBL,cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"You do not need to do anything during input preview.",InputPreviewInstructionLine1BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"This step is to allow you to confirm that all the input you have made is correct.",InputPreviewInstructionLine2BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Any incorrect input can be re-run through the menu.",InputPreviewInstructionLine3BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Press 'e' when you are done confirming your input to return to the menu.",InputPreviewInstructionLine4BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Press 'e' to continue to the Input Preview Screen.",InputPreviewInstructionLine5BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    while True:
        cv2.imshow(WindowName,GeneratedImage)
        key=cv2.waitKey(1)&0xFF
        if key==ord("e"):
            break
    return

def InputPreview(NonWritableImage,menuRefresher):
    if Show_Instructions:
        InputPreviewExplanation()
        
    scrollHeight=0
    scrollWidth=0
    
    cv2.createTrackbar("Hscroll", WindowName, scrollHeight, (height-Screen_Height),TrackbarChange)
    cv2.createTrackbar("Wscroll", WindowName, scrollWidth, (width-Screen_Width),TrackbarChange)
    
    WritableClone=NonWritableImage.copy()
    WritableCloneBackup=WritableClone.copy()
    
    Chairs=[]
    
    for chair in range(len(RoomInfo['Locs']['x'])):
        Chairs.append([int(RoomInfo['Locs']['x'][chair]),
                       int(RoomInfo['Locs']['y'][chair]),
                       int(RoomInfo['Locs']['type'][chair])])
        
    #Draw all chairs on a fresh room
    for chair in Chairs:
        cv2.circle(WritableClone,(chair[0],chair[1]),2,Red,2)
        PolygonPoints=RoomInfo['PolyPoints'][chair[2]]
        PolygonOffsets=RoomInfo['PolyPointOffsets'][chair[2]]
        for polygonPoint in range(len(PolygonPoints)):
            line=cv2.line(WritableClone,
                          (PolygonPoints[polygonPoint][0]+chair[0]+PolygonOffsets[0],
                           PolygonPoints[polygonPoint][1]+chair[1]+PolygonOffsets[1]),
                          (PolygonPoints[polygonPoint-1][0]+chair[0]+PolygonOffsets[0],
                           PolygonPoints[polygonPoint-1][1]+chair[1]+PolygonOffsets[1]),
                          Red,2)
    #Draws the Selected Scale
    for i in range(len(RoomInfo["ScaleSelectedPoints"])):
        line=cv2.line(WritableClone, RoomInfo["ScaleSelectedPoints"][i],RoomInfo["ScaleSelectedPoints"][i-1],Red,2)
    #Draws the rectangles for the selection of each chair type
    for i in range(len(RoomInfo["ChairTypes"])):#for each chair
        cds = getCoords(RoomInfo["ChairTypes"][i])
        rect=cv2.rectangle(WritableClone, (cds[0],cds[2]),(cds[1],cds[3]),Purple,2)
    while True:
        ScrollHeight=cv2.getTrackbarPos("Hscroll",WindowName)
        ScrollWidth=cv2.getTrackbarPos("Wscroll",WindowName)
        ShownImage=np.array([(ScrollWidth,ScrollHeight),
                             (ScrollWidth+width,ScrollHeight),
                             (ScrollWidth+width,ScrollHeight+height),
                             (ScrollWidth,ScrollHeight+height)])
        cds=getCoords(ShownImage)
        Image = WritableClone[cds[2]:cds[3], cds[0]:cds[1]]
        cv2.imshow(WindowName, Image)
        
        key=cv2.waitKey(1)&0xFF
        if key==ord('e'):
            break
    menuRefresher.append('r')

In [17]:
def SolveRoomExplanation():
    GeneratedImage=np.ones((Screen_Height,Screen_Width,1),np.uint8)*255
    GeneratedImage=cv2.cvtColor(GeneratedImage,cv2.COLOR_GRAY2RGB)
    GeneratedImage=cv2.putText(GeneratedImage,"Solve Room",SolveRoomInstructionTitleBL,cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"You do not need to do anything during the Solve Room step.",SolveRoomInstructionLine1BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"During this step the tool will take all your input and create a solution for the room.",SolveRoomInstructionLine2BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"When finished, the tool will save the solution as an image in the same folder it is located in.",SolveRoomInstructionLine3BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"The tool will automatically quit when completed.",SolveRoomInstructionLine4BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    GeneratedImage=cv2.putText(GeneratedImage,"Press 'e' to continue and solve the room.",SolveRoomInstructionLine5BL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
    while True:
        cv2.imshow(WindowName,GeneratedImage)
        key=cv2.waitKey(1)&0xFF
        if key==ord("e"):
            break
    return
def SolveRoom(NonWritableImage,menuRefresher):
    if Show_Instructions:
        SolveRoomExplanation()
    FinalInfo={"Chairs":[],"Polygons":[],"Middles":[],"In Solution":[]}
    
    for chair in range(len(RoomInfo['Locs']['x'])):
        FinalInfo["Chairs"].append([int(RoomInfo['Locs']['x'][chair]),
                                    int(RoomInfo['Locs']['y'][chair]),
                                    int(RoomInfo['Locs']['type'][chair])])
    
    for chair in FinalInfo["Chairs"]:
        FinalInfo["Polygons"].append(RoomInfo["Polys"][chair[2]])
    for chair in range(len(FinalInfo["Chairs"])):
        FinalInfo["Polygons"][chair]=shapely.affinity.translate(FinalInfo["Polygons"][chair],
                                                                xoff=FinalInfo["Chairs"][chair][0],
                                                                yoff=FinalInfo["Chairs"][chair][1])
        FinalInfo["Polygons"][chair]=shapely.affinity.translate(FinalInfo["Polygons"][chair], 
                                                                xoff=RoomInfo["PolyPointOffsets"][FinalInfo["Chairs"][chair][2]][0], 
                                                                yoff=RoomInfo["PolyPointOffsets"][FinalInfo["Chairs"][chair][2]][1])
    for polygon in FinalInfo["Polygons"]:
        FinalInfo["Middles"].append(polygon.centroid)

    # set axis tick size
    plt.rc('xtick', labelsize=4) 
    plt.rc('ytick', labelsize=4) 
    fig, ax = plt.subplots(dpi=2000)
    plt.axis('off')

    # set axis line size
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(0.5)
    
    # plot points    
    xs=[]
    ys=[]
    for chair in FinalInfo["Middles"]:
        xs.append(chair.x)
        ys.append(chair.y)
    #plt.scatter(xs, ys, s=0.3, color='blue', alpha=1)

    # generate edges from distances
    edgelist = list()      # overall list of (directed) edges

    for i in range(len(FinalInfo["Chairs"])):
        for j in range(len(FinalInfo["Chairs"])):
            if (i != j):
                dist = FinalInfo['Polygons'][j].distance(FinalInfo['Middles'][i])
                if (dist <= RoomInfo["PixelsToDistance"]):
                    edgelist.append((i,j)) # add edge if seat is too close

    
    nodes=[]
    for i in range(len(FinalInfo["Chairs"])):
        nodes.append(i)
        
    # solve problem
    sol = miset(nodes, edgelist, OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    FinalInfo["In Solution"]=list(sol.values())
    
    
    for i in range(len(FinalInfo["Chairs"])):
        if FinalInfo["In Solution"][i]==0: 
            xz,yz = FinalInfo["Polygons"][i].exterior.xy
            ax.plot(xz, yz, color='blue', alpha=1, linewidth=0.2, solid_capstyle='round', zorder=2)
            ax.fill(xz, yz, alpha=0.6, fc='lightblue', ec='darkblue', linewidth=0.15, zorder=2)
        else:
            xz,yz = FinalInfo["Polygons"][i].exterior.xy
            ax.plot(xz, yz, color='orange', alpha=1, linewidth=0.2, solid_capstyle='round', zorder=3)
            ax.fill(xz, yz, alpha=0.6, fc='orange', ec='darkred', linewidth=0.15, zorder=3)
            draw_circle = plt.Circle((FinalInfo['Middles'][i].x, 
                                      FinalInfo['Middles'][i].y), 
                                     RoomInfo["PixelsToDistance"], 
                                     fill=False, ec='darkblue', ls='--', lw=0.2, zorder=4)
            ax.add_artist(draw_circle)
    seats=0
    for i in sol.values():
        seats += i
    plt.text(10, 10, str(int(seats)) + " seats",
             fontsize=6, color='red',
             horizontalalignment='left',
             verticalalignment='top')
    ax.imshow(NonWritableImage)
    plt.savefig(solution_name+".jpg",orientation='portrait',format='jpg',dpi=2000)


In [18]:
#Main Menu:
def MainMenu():
    menuRefresher=['r']
    menuTasks=[]
    
    def get_menu_click_coords(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            
            #Detect click on Scale Selection Button
            if(x<ScaleSelectionButtonEndPoint[0] and x>ScaleSelectionButtonStartPoint[0]) and (y<ScaleSelectionButtonEndPoint[1] and y>ScaleSelectionButtonStartPoint[1]):
                menuTasks.append('Scale Selection')
            elif (x<ChairTypeSelectionButtonEndPoint[0] and x>ChairTypeSelectionButtonStartPoint[0]) and (y<ChairTypeSelectionButtonEndPoint[1] and y>ChairTypeSelectionButtonStartPoint[1]):
                menuTasks.append('Chair Type Selection')
            elif((x<PolygonCreationSelectionButtonEndPoint[0] and x>PolygonCreationSelectionButtonStartPoint[0]) and (y<PolygonCreationSelectionButtonEndPoint[1] and y>PolygonCreationSelectionButtonStartPoint[1])) and Completions["Chair Type Selection Status"]:
                menuTasks.append('Polygon Creation Selection')
            elif((x<ChairRecognitionSelectionButtonEndPoint[0] and x>ChairRecognitionSelectionButtonStartPoint[0]) and (y<ChairRecognitionSelectionButtonEndPoint[1] and y>ChairRecognitionSelectionButtonStartPoint[1])) and Completions["Chair Type Selection Status"]:
                menuTasks.append('Chair Recognition Selection')
            elif((x<ChairAdditionSelectionButtonEndPoint[0] and x>ChairAdditionSelectionButtonStartPoint[0]) and (y<ChairAdditionSelectionButtonEndPoint[1] and y>ChairAdditionSelectionButtonStartPoint[1]) and (Completions["Chair Recognition Status"] and Completions["Polygon Creation Status"])):
                menuTasks.append("Chair Addition Selection")
            elif((x<ChairDeletionSelectionButtonEndPoint[0] and x>ChairDeletionSelectionButtonStartPoint[0]) and (y<ChairDeletionSelectionButtonEndPoint[1] and y>ChairDeletionSelectionButtonStartPoint[1]) and (Completions["Chair Recognition Status"] and Completions["Polygon Creation Status"])):
                menuTasks.append("Chair Deletion Selection")
            elif((x<InputPreviewButtonEndPoint[0] and x>InputPreviewButtonStartPoint[0]) and (y<InputPreviewButtonEndPoint[1] and y>InputPreviewButtonStartPoint[1]) and (Completions["Polygon Creation Status"] and Completions["Chair Recognition Status"])):
                menuTasks.append("Input Preview")
            elif((x<SolveAndSaveButtonEndPoint[0] and x>SolveAndSaveButtonStartPoint[0]) and(y<SolveAndSaveButtonEndPoint[1] and y>SolveAndSaveButtonStartPoint[1]) and(Completions["Input Confirmation Status"])):
                menuTasks.append("Solve And Save")


    
    cv2.setMouseCallback(WindowName, get_menu_click_coords)
    
    while(True):
        if(len(menuRefresher)>0):
            
            GeneratedImage=np.ones((Screen_Height,Screen_Width,1),np.uint8)*255
            GeneratedImage=cv2.cvtColor(GeneratedImage,cv2.COLOR_GRAY2RGB)
            
            #Draw Button for Scale Selection
            if Completions["Scale Selection Status"]==False:
                ScaleSelectionButtonColor=ButtonNotCompleteColor
                ScaleSelectionStatus="Not Completed"
                ScaleSelectionStatusWritePoint=(ScaleSelectionButtonStartPoint[0]+NotCompletedXAdjustment,ScaleSelectionButtonBL[1])
            else:
                ScaleSelectionButtonColor=ButtonCompleteColor
                #ScaleSelectionStatus=str(RoomInfo["PixelsToDistance"])
                ScaleSelectionStatus="Completed"
                ScaleSelectionStatusWritePoint=(ScaleSelectionButtonStartPoint[0]+CompletedXAdjustment,ScaleSelectionButtonBL[1])
            GeneratedImage=cv2.rectangle(GeneratedImage,ScaleSelectionButtonStartPoint,ScaleSelectionButtonEndPoint,ScaleSelectionButtonColor,ButtonThickness)
            GeneratedImage=cv2.putText(GeneratedImage,"Scale pixels to distance:",ScaleSelectionButtonBL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
            GeneratedImage=cv2.putText(GeneratedImage,ScaleSelectionStatus,ScaleSelectionStatusWritePoint,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
            #Draw Button for Chair Type Selection
            if Completions["Chair Type Selection Status"]==False:
                ChairTypeSelectionButtonColor=ButtonNotCompleteColor
                ChairTypeSelectionStatus="Not Completed"
                ChairTypeSelectionStatusWritePoint=(ChairTypeSelectionButtonStartPoint[0]+NotCompletedXAdjustment,ChairTypeSelectionButtonBL[1])
            else:
                ChairTypeSelectionButtonColor=ButtonCompleteColor
                ChairTypeSelectionStatus="Completed"
                ChairTypeSelectionStatusWritePoint=(ChairTypeSelectionButtonStartPoint[0]+CompletedXAdjustment,ChairTypeSelectionButtonBL[1])
            GeneratedImage=cv2.rectangle(GeneratedImage,ChairTypeSelectionButtonStartPoint,ChairTypeSelectionButtonEndPoint,ChairTypeSelectionButtonColor,ButtonThickness)
            GeneratedImage=cv2.putText(GeneratedImage,"Chair Type Selection:",ChairTypeSelectionButtonBL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
            GeneratedImage=cv2.putText(GeneratedImage,ChairTypeSelectionStatus,ChairTypeSelectionStatusWritePoint,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
            #Draw buttons that rely on chair type selection if chair type selection has been completed.
            if Completions["Chair Type Selection Status"]:
                
                #Draw button for Polygon Creation
                if Completions["Polygon Creation Status"]==False:
                    PolygonCreationButtonColor=ButtonNotCompleteColor
                    PolygonCreationStatus="Not Completed"
                    PolygonCreationSelectionStatusWritePoint=(PolygonCreationSelectionButtonStartPoint[0]+NotCompletedXAdjustment,PolygonCreationSelectionButtonBL[1])
                else:
                    PolygonCreationButtonColor=ButtonCompleteColor
                    PolygonCreationStatus="Completed"
                    PolygonCreationSelectionStatusWritePoint=(PolygonCreationSelectionButtonStartPoint[0]+CompletedXAdjustment,PolygonCreationSelectionButtonBL[1])
                GeneratedImage=cv2.rectangle(GeneratedImage,PolygonCreationSelectionButtonStartPoint,PolygonCreationSelectionButtonEndPoint,PolygonCreationButtonColor,ButtonThickness)
                GeneratedImage=cv2.putText(GeneratedImage,"Polygon Creation:",PolygonCreationSelectionButtonBL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
                GeneratedImage=cv2.putText(GeneratedImage,PolygonCreationStatus,PolygonCreationSelectionStatusWritePoint,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
                #Draw button for Chair recognition
                if Completions["Polygon Creation Status"]:
                    if Completions["Chair Recognition Status"]==False:
                        ChairRecognitionButtonColor=ButtonNotCompleteColor
                        ChairRecognitionStatus="Not Completed"
                        ChairRecognitionSelectionStatusWritePoint=(ChairRecognitionSelectionButtonStartPoint[0]+NotCompletedXAdjustment,ChairRecognitionSelectionButtonBL[1])
                    else:
                        ChairRecognitionButtonColor=ButtonCompleteColor
                        ChairRecognitionStatus="Completed"
                        ChairRecognitionSelectionStatusWritePoint=(ChairRecognitionSelectionButtonStartPoint[0]+CompletedXAdjustment,ChairRecognitionSelectionButtonBL[1])
                    GeneratedImage=cv2.rectangle(GeneratedImage,ChairRecognitionSelectionButtonStartPoint,ChairRecognitionSelectionButtonEndPoint,ChairRecognitionButtonColor,ButtonThickness)
                    GeneratedImage=cv2.putText(GeneratedImage,"Chair Recognition:",ChairRecognitionSelectionButtonBL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
                    GeneratedImage=cv2.putText(GeneratedImage,ChairRecognitionStatus,ChairRecognitionSelectionStatusWritePoint,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
                #Draw buttons that rely on chair recognition and Polygon creation if they have both been completed.
                    if Completions["Chair Recognition Status"]:
                        #Draw button for Chair Addition
                        if Completions["Chair Addition Status"]==False:
                            ChairAdditionButtonColor=ButtonNotCompleteColor
                            ChairAdditionStatus="Not Completed"
                            ChairAdditionSelectionStatusWritePoint=(ChairAdditionSelectionButtonStartPoint[0]+NotCompletedXAdjustment,ChairAdditionSelectionButtonBL[1])
                        else:
                            ChairAdditionButtonColor=ButtonCompleteColor
                            ChairAdditionStatus="Completed"
                            ChairAdditionSelectionStatusWritePoint=(ChairAdditionSelectionButtonStartPoint[0]+CompletedXAdjustment,ChairAdditionSelectionButtonBL[1])
                        GeneratedImage=cv2.rectangle(GeneratedImage,ChairAdditionSelectionButtonStartPoint,ChairAdditionSelectionButtonEndPoint,ChairAdditionButtonColor,ButtonThickness)
                        GeneratedImage=cv2.putText(GeneratedImage,"Chair Addition:",ChairAdditionSelectionButtonBL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
                        GeneratedImage=cv2.putText(GeneratedImage,ChairAdditionStatus,ChairAdditionSelectionStatusWritePoint,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
                        #Draw button for Chair Deletion
                        if Completions["Chair Deletion Status"]==False:
                            ChairDeletionButtonColor=ButtonNotCompleteColor
                            ChairDeletionStatus="Not Completed"
                            ChairDeletionSelectionStatusWritePoint=(ChairDeletionSelectionButtonStartPoint[0]+NotCompletedXAdjustment,ChairDeletionSelectionButtonBL[1])
                        else:
                            ChairDeletionButtonColor=ButtonCompleteColor
                            ChairDeletionStatus="Completed"
                            ChairDeletionSelectionStatusWritePoint=(ChairDeletionSelectionButtonStartPoint[0]+CompletedXAdjustment,ChairDeletionSelectionButtonBL[1])
                        GeneratedImage=cv2.rectangle(GeneratedImage,ChairDeletionSelectionButtonStartPoint,ChairDeletionSelectionButtonEndPoint,ChairDeletionButtonColor,ButtonThickness)
                        GeneratedImage=cv2.putText(GeneratedImage,"Chair Deletion:",ChairDeletionSelectionButtonBL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
                        GeneratedImage=cv2.putText(GeneratedImage,ChairDeletionStatus,ChairDeletionSelectionStatusWritePoint,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
                        #Draw button for Input Confirmation
                        if Completions["Input Confirmation Status"]==False:
                            InputPreviewButtonColor=ButtonNotCompleteColor
                            InputConfirmationStatus="Not Completed"
                            InputConfirmationSelectionStatusWritePoint=(InputPreviewButtonStartPoint[0]+NotCompletedXAdjustment,InputPreviewButtonBL[1])
                        else:
                            InputPreviewButtonColor=ButtonCompleteColor
                            InputConfirmationStatus="Completed"
                            InputConfirmationSelectionStatusWritePoint=(InputPreviewButtonStartPoint[0]+CompletedXAdjustment,InputPreviewButtonBL[1])
                        GeneratedImage=cv2.rectangle(GeneratedImage,InputPreviewButtonStartPoint,InputPreviewButtonEndPoint,InputPreviewButtonColor,ButtonThickness)
                        GenerateImage=cv2.putText(GeneratedImage,"Input Confirmation:",InputPreviewButtonBL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
                        GeneratedImage=cv2.putText(GeneratedImage,InputConfirmationStatus,InputConfirmationSelectionStatusWritePoint,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
                        if Completions["Input Confirmation Status"]:
                            SolveAndSaveButtonColor=ButtonNotCompleteColor
                            SolveAndSaveSelectionStatusWritePoint=(SolveAndSaveButtonStartPoint[0]+NotCompletedXAdjustment,SolveAndSaveButtonBL[1])
                            GeneratedImage=cv2.rectangle(GeneratedImage,SolveAndSaveButtonStartPoint,SolveAndSaveButtonEndPoint,SolveAndSaveButtonColor,ButtonThickness)
                            GenerateImage=cv2.putText(GeneratedImage,"Solve and Save Room:",SolveAndSaveButtonBL,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
                            GeneratedImage=cv2.putText(GeneratedImage,"Not Completed",SolveAndSaveSelectionStatusWritePoint,cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,0),1,cv2.LINE_AA)
        menuRefresher=[]
        cv2.imshow(WindowName,GeneratedImage)
        
        if(len(menuTasks)>0):
            if menuTasks[0]=='Scale Selection':
                ScaleSelection(NonWritableImage,menuRefresher)
                Completions["Scale Selection Status"]=True
                menuTasks=[]
            elif menuTasks[0]=='Chair Type Selection':
                Completions["Polygon Creation Status"]=False
                Completions["Chair Recognition Status"]=False
                Completions["Chair Addition Status"]=False
                Completions["Chair Deletion Status"]=False
                ChairTypeSelection(NonWritableImage,menuRefresher)
                Completions["Chair Type Selection Status"]=True
                menuTasks=[]
            elif menuTasks[0]=='Polygon Creation Selection':
                Completions["Chair Recognition Status"]=False
                Completions["Chair Addition Status"]=False
                Completions["Chair Deletion Status"]=False
                PolygonCreation(NonWritableImage,menuRefresher,RoomInfo)
                Completions["Polygon Creation Status"]=True
                menuTasks=[]
            elif menuTasks[0]=='Chair Recognition Selection':
                Completions["Chair Addition Status"]=False
                Completions["Chair Deletion Status"]=False
                ChairRecognition(NonWritableImage,menuRefresher,RoomInfo["ChairTypes"])
                Completions["Chair Recognition Status"]=True
                menuTasks=[]
            elif menuTasks[0]=='Chair Addition Selection':
                ChairAddition(NonWritableImage,menuRefresher,RoomInfo["ChairTypes"])
                Completions["Chair Addition Status"]=True
                menuTasks=[]
            elif menuTasks[0]=='Chair Deletion Selection':
                ChairDeletion(NonWritableImage,menuRefresher)
                Completions["Chair Deletion Status"]=True
                menuTasks=[]
            elif menuTasks[0]=='Input Preview':
                InputPreview(NonWritableImage,menuRefresher)
                Completions["Input Confirmation Status"]=True
                menuTasks=[]
            elif menuTasks[0]=='Solve And Save':
                SolveRoom(NonWritableImage,menuRefresher)
                menuTasks=[]
                cv2.destroyAllWindows()
                break
                
            
        else:
            cv2.setMouseCallback(WindowName,get_menu_click_coords)
        key = cv2.waitKey(1) & 0xFF
    
        # if the 'e' key is pressed, break from the loop
        if key == ord("e"):
            cv2.destroyAllWindows()
            break
MainMenu()

#Better Font for the text-Hershey Simplex actually looks the best to me out of the provided fonts.
#Maybe add ability to modify configs from a different file
